In [1]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('data.csv') #Three stocks (R,X_s,X_b) Without predictors

In [3]:
#Initialize Neural Network and set-up the placeholders
tf.reset_default_graph()
NN_input = tf.placeholder(shape=[1,2],dtype=tf.float32)
NN_weights = tf.Variable(tf.random_uniform([2,10],0,0.01))
Q_FA = tf.matmul(NN_input,NN_weights)
A_Max = tf.argmax(Q_FA,1)

# Calculate loss for the NN from the Q values
Q_Next = tf.placeholder(shape=[1,10],dtype=tf.float32)
loss = tf.reduce_sum(tf.square(Q_Next - Q_FA))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

#Define Action Matrix (Now discrete case) 
A = np.linspace(0,1,10) # portfolio weights of stocks (1-weight) is the weight in the bonds

In [ ]:
# initialize
init = tf.global_variables_initializer()
gamma = 1 
epsilon = 0.1
train_data = 241 # also an expanding window
jList = []
TWlistTrain = []
TWlist = []
Index = []
MWeights = []
# data parsing
dates = data['Date']
mdata = data[['r','xs','xb']]
mdata.index = pd.DatetimeIndex(dates)
n = data.size/4-4
periods = 60
epochs = 20

with tf.Session() as sess:
    for i in range(247,int(n-periods-1)):
        OptimalWeights = np.zeros(periods-1)
        currentK = 0
        print(i)
        
        while currentK < periods - 1:
            #Initilization
            NN_data = mdata[i-200:i+currentK]
            sess.run(init) # initialize the Neural Network again
            rAll = 0
            currentEpoch = 0
            
            while currentEpoch < epochs:
                #Training of the Q-Network for the data available (with Neural Nets) 
                for j in range(0,len(NN_data)):
                    s = NN_data.iloc[j,1:4].values.reshape(1,2)
                    #Choose an action by greedily (with e chance of random action) from the Q-network
                    a_int,allQ = sess.run([A_Max,Q_FA],feed_dict={NN_input:s})
                    a = A[a_int-1]  # -1 because the output neurons are labeled 1 till 101 and it will be an index
                    if np.random.rand(1) < epsilon:
                        a = random.choice(A)

                    #Get new state and reward from environment
                    s1 = data.iloc[j+i,2:4].values.reshape(1,2)
                    r = (a*s1[0][0] + (1-a)*s1[0][1]) #reward: this is now the wealth gained from this step, but could be other rewards like utility
                    Q = sess.run(Q_FA,feed_dict={NN_input:s1})

                    #Obtain maxQ' and set our target value for chosen action.
                    Q1 = np.max(Q)
                    targetQ = allQ
                    targetQ[0,a_int] = r + gamma*Q1

                    #Train the neural network using target and predicted Q values
                    _,W1 = sess.run([updateModel,NN_input],feed_dict={NN_input:s,Q_Next:targetQ})
                    rAll += r
                    s = s1
                    if currentEpoch  > 5:
                        # decrease amount of random actions over time in order to improve exploitation rather than exploration
                        # only increase exploitation when a good action has been found (otherwise one exploits a bad solution)
                        e = 1./((i/50) + 10)
                        break
                currentEpoch += 1
        
            # After training now calculate the optimal weights for the K=60 periods to come
            s = mdata.iloc[i+periods,1:4].values.reshape(1,2)
            a_int,allQ = sess.run([A_Max,Q_FA],feed_dict={NN_input:s})
            aOpt = A[a_int-1]
            OptimalWeights[currentK] = aOpt
            currentK += 1
        
        # For insight purposes
        MWeights.append(np.mean(OptimalWeights))
        TWlist.append(np.exp(sum(OptimalWeights*mdata[i+1:i+currentK+1]['xs'] + (1-OptimalWeights)*mdata[i+1:i+currentK+1]['xb'])))
        Index.append(i)
        
        print('Writing away results')
        df = pd.DataFrame({'index date':Index,'TW':TWlist, 'Mean Weights Xs':MWeights})
        df.to_excel('Results_NN_g10.xlsx', sheet_name='sheet1', index=False)
    # close session
plt.plot(MWeights)
plt.plot(TWlist)
        

247
Writing away results
248
Writing away results
249
Writing away results
250
Writing away results
251
Writing away results
252
Writing away results
253
Writing away results
254
Writing away results
255
Writing away results
256
Writing away results
257
Writing away results
258
Writing away results
259
Writing away results
260
Writing away results
261
Writing away results
262
Writing away results
263
Writing away results
264
Writing away results
265
Writing away results
266
Writing away results
267
Writing away results
268
Writing away results
269
Writing away results
270
Writing away results
271
Writing away results
272
Writing away results
273
Writing away results
274
Writing away results
275
Writing away results
276
Writing away results
277
Writing away results
278
Writing away results
279
Writing away results
280


In [ ]:
plt.plot(MWeights)
plt.plot(TWlist)

In [25]:
print(mdata[i+1:i+currentK+1]['xs'])
print(optimalweights.size)
TWlist.append(np.exp(sum(optimalweights*mdata[i+1:i+currentK+1]['xs'] + (1-optimalweights)*mdata[i+1:i+currentK+1]['xb'])))


Date
1974-04-01   -0.052899
1974-05-01   -0.048840
1974-06-01   -0.029525
1974-07-01   -0.077171
1974-08-01   -0.095223
1974-09-01   -0.116435
1974-10-01    0.159368
1974-11-01   -0.047272
1974-12-01   -0.032963
1975-01-01    0.136383
1975-02-01    0.053828
1975-03-01    0.025616
1975-04-01    0.041822
1975-05-01    0.050782
1975-06-01    0.047023
1975-07-01   -0.065493
1975-08-01   -0.028924
1975-09-01   -0.043360
1975-10-01    0.050890
1975-11-01    0.026656
1975-12-01   -0.015531
1976-01-01    0.121953
1976-02-01    0.003247
1976-03-01    0.022632
1976-04-01   -0.014439
1976-05-01   -0.013702
1976-06-01    0.039979
1976-07-01   -0.010515
1976-08-01   -0.005718
1976-09-01    0.020251
1976-10-01   -0.024473
1976-11-01    0.001406
1976-12-01    0.057966
1977-01-01   -0.040187
1977-02-01   -0.019652
1977-03-01   -0.013079
1977-04-01    0.001169
1977-05-01   -0.014918
1977-06-01    0.047184
1977-07-01   -0.017135
1977-08-01   -0.017918
1977-09-01   -0.003268
1977-10-01   -0.044413
1977-1

ValueError: operands could not be broadcast together with shapes (60,) (59,) 